# Imblanced Classification

This notebook was created as part of a project to predict which members of a not for profit orginizaiton will renew their membership and which will cancel.  Getting useful results from an imbalanced data set like this one can be challenging.  Different methodologies can be applied to try and overcome this issue.    In addtion upsampling techniques are in the random Forest notebook and logistic regression notebook.  Smoteen is in the logistic regression notebook and one class SVM is in the SVM notebook.  Balanced weight  parameter for each type of model is in the notebooks for each respective model.  The results were not what we were hoping for but this is a very useful collection of many great techniques.  

### try different classification thresholds
We can change the threshold the model uses to decide whether the member is likely to cancel or renew.  The default threshold is 50%.  There are different ways to calculate what maybe a more appropriate threshold. This code is from  from https://machinelearningmastery.com/

### Notes
much of the code was adapted from https://machinelearningmastery.com/ 
some of the code including the upsampling and learning curves was adapted from the machine learning with python github repo
various other resources also provided parts of the code.
The code was edited to hide all proprietary information.

### Packages

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import fbeta_score, make_scorer, f1_score, balanced_accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split

### Import Data

In [4]:
data = pd.read_csv("format_variables_output4.csv")
#data.head()

In [4]:
# drop the entries for new members who joined leavig renewals and cancelations only
data = data[data['Target']!=2]

show the balance of the data set the renewal variable (1) has over 90% while cancelations are only 9.2%
This imbalance in the data set will be a challenge of this predicitive model.

In [5]:
data['Target'].value_counts(normalize=True)

1    0.907453
0    0.092547
Name: Target, dtype: float64

In [6]:
data.columns

Index(['Unnamed: 0', 'MemberName', 'NF_Year', 'ACS_Year', 'JoinDate',
       'ExpireDate', 'MemberStatus', 'State', 'StateName', 'StateCode',
       'Region', 'SummaryLevel', 'CombinedFips', 'Target', 'TotalPopulation',
       'MedianHouseholdIncome', 'MedianAge', 'NonHispanicWhites',
       'TotalWorkers', 'UseCar', 'PopOver25', 'Bachelors', 'Agriculture',
       'Construction', 'Manufacturing', 'WholesaleTrade', 'Retail',
       'Transportation', 'Information', 'Finance', 'Professional', 'Education',
       'Arts', 'OtherServices', 'PublicAdministration', 'ArmedForces',
       'Years as Member', '1', '10', '12', '13', '15', '16', '17', '18', '19',
       '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30',
       '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41',
       '42', '44', '45', '46', '47', '48', '49', '5', '50', '51', '53', '54',
       '55', '56', '6', '8', '9', 'Pop_Den', 'binned_pop_den',
       'MedianIncomeAdjusted', '%NonHispanicW

In [5]:
data1 = data[['Target','TotalPopulation', 'MedianAge', 'MedianIncomeAdjusted', '%NonHispanicWhites',
            '%DrivetoWork','%Bachelors','Pop_Den']]
# data1.head()

In [ ]:
# split into training and testing sets the classifier and its hyperparameters were selected in a different notebook
X =  data1.drop(columns=['Target']).values
y = data1['Target'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)
clf = RandomForestClassifier(n_estimators = 100, random_state = 0, min_samples_leaf=2, min_samples_split =2, criterion =  'entropy', 
                             max_depth= None, max_features = 'log2', )

### try different classification thresholds
We can change the threshold the model uses to decide whether the member is likely to cancel or renew.  The default threshold is 50%.  There are different ways to calculate what maybe a more appropriate threshold. This code is from  from https://machinelearningmastery.com/

In [53]:
from sklearn.metrics import roc_curve

model = clf    
model.fit(X_train, y_train)
# predict probabilities
yhat = model.predict_proba(X_test)
# keep probabilities for the positive outcome only
yhat = yhat[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(y_test, yhat)
# calculate the g-mean for each threshold this takes into account the balance of specetivity and sensitivity 
gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold with Youden J=%f' % (best_thresh))

Best Threshold=0.924667, G-Mean=0.593
Best Threshold with Youden J=0.927714


In [54]:
precision, recall, thresholds = precision_recall_curve(y_test, yhat)
# convert to f score
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))

Best Threshold=0.475333, F-Score=0.951


In [57]:
# evaluate the 3 different thresholds to see which has the highest f1 score

# keep probabilities for the positive outcome only
probs = yhat
# define thresholds
thresholds = [.475,.924667,927714]
# evaluate each threshold
scores = [f1_score(y_test, to_labels(probs, t)) for t in thresholds]
# get best threshold
ix = np.argmax(scores)
print(scores)
print('Threshold=%.3f, F-Score=%.5f' % (thresholds[ix], scores[ix]))

[0.9509563511525257, 0.7052783571313973, 0.0]
Threshold=0.475, F-Score=0.95096


In [66]:
# Define a function which will apply threshold to positive probabilities to create labels of 
# of classified as 0 or 1 depending on if it is above or bleow the custom threshold
def to_labels(pos_probs, threshold):
    return (pos_probs >= threshold).astype('int') # true will return 1 false return 0

Lets try different thresholds to see if we can create a better model.

In [59]:
print(classification_report(y_test, to_labels(probs, .475))) 

              precision    recall  f1-score   support

           0       0.75      0.02      0.04       406
           1       0.91      1.00      0.95      3881

    accuracy                           0.91      4287
   macro avg       0.83      0.51      0.50      4287
weighted avg       0.89      0.91      0.86      4287



In [60]:
print(classification_report(y_test, to_labels(probs, .924))) 

              precision    recall  f1-score   support

           0       0.13      0.61      0.21       406
           1       0.93      0.57      0.71      3881

    accuracy                           0.57      4287
   macro avg       0.53      0.59      0.46      4287
weighted avg       0.86      0.57      0.66      4287



Lower thresholds mean less negative labels which raises the precision on the cancelations and lowers the recall.  A higher threshold increases the recall as it is harder to get a positive label but of course the precision is lower.  The total f1 score for negatives is improved but the precision has droped too low. 

In [63]:
print(classification_report(y_test, to_labels(probs, .85))) 

              precision    recall  f1-score   support

           0       0.15      0.30      0.20       406
           1       0.92      0.83      0.87      3881

    accuracy                           0.78      4287
   macro avg       0.54      0.56      0.54      4287
weighted avg       0.85      0.78      0.81      4287



#### Conclusion:
Changing the threshold didn't solve the problem and produce a useful model.  It may be something to come back to and combine with other methods and models if we can get a pretty good model this is a way to tweak the results,

## Resample the data set to deal with class imbalance with upsampling

In [6]:
data1['Target'].value_counts()

1    11786
0     1202
Name: Target, dtype: int64

In [7]:
X_imb =  data1.drop(columns=['Target']).values
y_imb = data1['Target'].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_imb, y_imb , test_size=0.33, random_state=42)

The split was done before the upsample. The upsampling was done only on the training data not the testing data.

In [9]:
from sklearn.utils import resample

print('Number of class 0 samples before:', X_train[y_train == 0].shape[0])

X_upsampled, y_upsampled = resample(X_train[y_train == 0],
                                    y_train[y_train == 0],
                                    replace=True,
                                    n_samples=X_train[y_train == 1].shape[0],
                                    random_state=123)

print('Number of class 0 samples after:', X_upsampled.shape[0])

Number of class 0 samples before: 802
Number of class 0 samples after: 7899


In [10]:
# after increasing the number of cancelations (negative outcome) the training data is merged with the renewals (positive outcome). 
X_train = np.vstack((X_train[y_train == 1], X_upsampled))
y_train = np.hstack((y_train[y_train == 1], y_upsampled))

In [11]:
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)
print(confusion_matrix(y_test,rfc_pred))
print(classification_report(y_test,rfc_pred))

[[  38  362]
 [  92 3795]]
              precision    recall  f1-score   support

           0       0.29      0.10      0.14       400
           1       0.91      0.98      0.94      3887

    accuracy                           0.89      4287
   macro avg       0.60      0.54      0.54      4287
weighted avg       0.86      0.89      0.87      4287



This gives us more balance between recall and precision but not a great model.  In the Random Forest notebook this model is tweaked.  

### Down Sampling
Lets try other techniques for imbalanced classification.

In [29]:
target = 'Target'
X =  data1.drop(columns=['Target']).values
y = data1['Target'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

In [30]:
from sklearn.utils import resample

print('Number of class 1 samples before:', X_train[y_train == 1].shape[0])

X_downsampled, y_downsampled = resample(X_train[y_train == 1],
                                    y_train[y_train == 1],
                                    replace=False,
                                    n_samples=X_train[y_train == 0].shape[0],
                                    random_state=123)

print('Number of class 0 samples after:', X_downsampled.shape[0])

Number of class 1 samples before: 7905
Number of class 0 samples after: 796


In [31]:
X_train = np.vstack((X_train[y_train == 0], X_downsampled))
y_train = np.hstack((y_train[y_train == 0], y_downsampled))

In [32]:
rfc = RandomForestClassifier(n_estimators=50)
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)
print(confusion_matrix(y_test,rfc_pred))
print(classification_report(y_test,rfc_pred))

[[ 241  165]
 [1790 2091]]
              precision    recall  f1-score   support

           0       0.12      0.59      0.20       406
           1       0.93      0.54      0.68      3881

    accuracy                           0.54      4287
   macro avg       0.52      0.57      0.44      4287
weighted avg       0.85      0.54      0.64      4287



#### Conclusion:
We  didn't get great precision with the minority class and we have worse performance with the majority class.  The overall f1 is 54.  The dataset is too small to downsample.  

## Smote (synthetic production of samples)

In [33]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

In [34]:
X =  data1.drop(columns=['Target']).values
y = data1['Target'].values
model = RandomForestClassifier()
# evaluate without smote using the auc roc as criteria for scoring
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC: %.3f' % np.mean(scores))

Mean ROC AUC: 0.642


In [35]:
steps = [('over', SMOTE()), ('model', RandomForestClassifier())]
pipeline = Pipeline(steps=steps)
# evaluate with smote 
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(pipeline, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC: %.3f' % np.mean(scores))

Mean ROC AUC: 0.665


smote did help but not by much lets run it on a model.

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=4)

In [37]:
# fit smote on the training data not the testing data
sm = SMOTE(random_state = 2) 
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel()) 

In [38]:
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape)) 
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape)) 
  
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1))) 
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0))) 


After OverSampling, the shape of train_X: (15770, 7)
After OverSampling, the shape of train_y: (15770,) 

After OverSampling, counts of label '1': 7885
After OverSampling, counts of label '0': 7885


In [39]:
rfc = RandomForestClassifier(n_estimators=50)
rfc.fit(X_train_res, y_train_res)
rfc_pred = rfc.predict(X_test)

In [40]:
print(confusion_matrix(y_test,rfc_pred))

[[ 100  286]
 [ 460 3441]]


In [41]:
print(classification_report(y_test,rfc_pred))

              precision    recall  f1-score   support

           0       0.18      0.26      0.21       386
           1       0.92      0.88      0.90      3901

    accuracy                           0.83      4287
   macro avg       0.55      0.57      0.56      4287
weighted avg       0.86      0.83      0.84      4287



the results aren't improved enough to be helpful but the official research recomends combining smote with downsampling.

In [48]:
from sklearn.utils import resample

print('Number of class 1 samples before:', X_train[y_train == 1].shape[0])

X_downsampled, y_downsampled = resample(X_train[y_train == 1],
                                    y_train[y_train == 1],
                                    replace=False,
                                    n_samples= 6000,
                                    random_state=123)

print('Number of class 1 samples after:', X_downsampled.shape[0])

Number of class 1 samples before: 7885
Number of class 1 samples after: 6000


In [49]:
X_train = np.vstack((X_train[y_train == 0], X_downsampled))
y_train = np.hstack((y_train[y_train == 0], y_downsampled))

In [51]:
sm = SMOTE(random_state = 2) 
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)   

In [52]:
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape)) 
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape)) 
  
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1))) 
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0))) 


After OverSampling, the shape of train_X: (12000, 7)
After OverSampling, the shape of train_y: (12000,) 

After OverSampling, counts of label '1': 6000
After OverSampling, counts of label '0': 6000


In [53]:
rfc = RandomForestClassifier(n_estimators=50)
rfc.fit(X_train_res, y_train_res)
rfc_pred = rfc.predict(X_test)
print(confusion_matrix(y_test,rfc_pred))
print(classification_report(y_test,rfc_pred))

[[ 101  285]
 [ 604 3297]]
              precision    recall  f1-score   support

           0       0.14      0.26      0.19       386
           1       0.92      0.85      0.88      3901

    accuracy                           0.79      4287
   macro avg       0.53      0.55      0.53      4287
weighted avg       0.85      0.79      0.82      4287



In [54]:
rfc = RandomForestClassifier(n_estimators=500)
rfc.fit(X_train_res, y_train_res)
rfc_pred = rfc.predict(X_test)
print(confusion_matrix(y_test,rfc_pred))
print(classification_report(y_test,rfc_pred))

[[  94  292]
 [ 556 3345]]
              precision    recall  f1-score   support

           0       0.14      0.24      0.18       386
           1       0.92      0.86      0.89      3901

    accuracy                           0.80      4287
   macro avg       0.53      0.55      0.53      4287
weighted avg       0.85      0.80      0.82      4287



### Conclusion:

Smote isn't a quick fix to the challenge.  If there was more time the data after smote and partial downsampling could be optimized with a grid search and evaluated with a learning curve.  The results were slightly better than with the upsampling.  There are still plenty of ways to try improving the model.  

###   imblearn automatic balance

This is a mehtod of imbalanced classification where weights are balanced to penalize wrong predictions of the minority class.

In [55]:
X =  data1.drop(columns=['Target']).values
y = data1['Target'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

In [56]:
from imblearn.ensemble import BalancedRandomForestClassifier


                           
clf = BalancedRandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train) 

print(clf.feature_importances_)  

rfc_pred = clf.predict(X_test)
print(confusion_matrix(y_test,rfc_pred))
print(classification_report(y_test,rfc_pred))

[0.31839366 0.04858149 0.14584345 0.14208609 0.08401095 0.10735835
 0.15372601]
[[ 202  204]
 [1383 2498]]
              precision    recall  f1-score   support

           0       0.13      0.50      0.20       406
           1       0.92      0.64      0.76      3881

    accuracy                           0.63      4287
   macro avg       0.53      0.57      0.48      4287
weighted avg       0.85      0.63      0.71      4287



In [59]:
# class balanced random forest for imbalanced classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
# define model
model = RandomForestClassifier(n_estimators=100, class_weight='balanced')
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.3f' % np.mean(scores))

Mean ROC AUC: 0.617


In [60]:
model.fit(X_train, y_train) 


rfc_pred = model.predict(X_test)
print(confusion_matrix(y_test,rfc_pred))
print(classification_report(y_test,rfc_pred))

[[  21  385]
 [   9 3872]]
              precision    recall  f1-score   support

           0       0.70      0.05      0.10       406
           1       0.91      1.00      0.95      3881

    accuracy                           0.91      4287
   macro avg       0.80      0.52      0.52      4287
weighted avg       0.89      0.91      0.87      4287



### Result
The Roc score of .617 was lower than what was achieved above with smote and the default random Forest model.  However the precison was much better than with the unweighted models run earlier.  The recall is still pretty low with upsampling we were able to get it to 15%.  

## Isolation forest  and one class classification
to deal with the weight imbalance of our dataset we can try anamoly detection to train a model on only the majority class and let the model attempt to detect any entries which are very different from the majority class.  This is another techniques used for imbalanced classification.  Several are tried here and one class SVM is in notebook 10.

In [61]:
from sklearn.ensemble import IsolationForest

In [62]:
data = pd.read_csv("format_variables_output4.csv")
data = data[data['Target']!=2]
data1 = data[['Target','TotalPopulation', 'MedianAge', 'MedianIncomeAdjusted', '%NonHispanicWhites',
            '%DrivetoWork','%Bachelors','Pop_Den']]


In [63]:
X_train = data1[data1['Target'] == 1]
X_outliers = data1[data1['Target'] == 0]
X_train =  X_train .drop(columns=['Target']).values
X_outliers = X_outliers.drop(columns=['Target']).values

In [64]:
X_outliers

array([[3.18970000e+04, 3.73000000e+01, 8.62020000e+04, ...,
        8.23887118e-01, 2.24009615e-01, 1.16000000e+01],
       [5.22900000e+03, 3.98000000e+01, 5.75100000e+04, ...,
        7.51542575e-01, 1.89542484e-01, 3.61800000e+02],
       [7.64000000e+02, 4.16000000e+01, 5.77040000e+04, ...,
        4.50363196e-01, 1.39534884e-01, 1.26500000e+02],
       ...,
       [3.21330000e+04, 4.25000000e+01, 5.46940000e+04, ...,
        9.29135338e-01, 1.71725677e-01, 5.70880000e+03],
       [1.04290000e+04, 4.04000000e+01, 7.79340000e+04, ...,
        9.34215168e-01, 1.79156745e-01, 2.01500000e+02],
       [3.17400000e+03, 4.37000000e+01, 5.69320000e+04, ...,
        8.65715983e-01, 1.33936262e-01, 6.93000000e+01]])

In [65]:
# training the model

# contamination paramater is set to the proportion of outliers 
clf = IsolationForest(contamination=0.091, behaviour='new')
clf.fit(X_train)

# predictions
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
y_pred_outliers = clf.predict(X_outliers)

C:\Users\brody\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  FutureWarning


In [66]:
# new, 'normal' observations ----
print("Accuracy:", list(y_pred_test).count(1)/y_pred_test.shape[0])


Accuracy: 0.9099603452297644


In [67]:
# outliers ----
print("Accuracy:", list(y_pred_outliers).count(-1)/y_pred_outliers.shape[0])


Accuracy: 0.09068219633943428


In [68]:
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.33, random_state=2, stratify=y)
# define outlier detection model
model = IsolationForest(contamination=0.091, behaviour='new')
# fit on majority class
trainX = trainX[trainy==1]
model.fit(trainX)
# detect outliers in the test set
yhat = model.predict(testX)
# mark inliers 1, outliers -1
testy[testy == 1] = 1
testy[testy == 0] = -1
# calculate score
score = f1_score(testy, yhat, pos_label=-1)
print('F1 Score: %.3f' % score)

C:\Users\brody\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  FutureWarning


F1 Score: 0.099


isolation forest did not predict many outliers we can detect 9% by guessing randomly.

### Local Outlier Factor to predict outliers

In [69]:
data = pd.read_csv("format_variables_output4.csv")
data = data[data['Target']!=2]
data1 = data[['Target','TotalPopulation', 'MedianAge', 'MedianIncomeAdjusted', '%NonHispanicWhites',
            '%DrivetoWork','%Bachelors','Pop_Den']]
X =  data1.drop(columns=['Target']).values
y = data1['Target'].values

In [70]:
from numpy import vstack
from sklearn.metrics import f1_score
from sklearn.neighbors import LocalOutlierFactor
 
# make a prediction with a lof model
def lof_predict(model, trainX, testX):
	# create one large dataset
	composite = vstack((trainX, testX))
	# make prediction on composite dataset
	yhat = model.fit_predict(composite)
	# return just the predictions on the test set
	return yhat[len(trainX):]
 
# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.33, random_state=2, stratify=y)
# define outlier detection model 
model = LocalOutlierFactor(contamination=0.091)  # contamination is the proportion of outliers
# get examples for just the majority class
trainX = trainX[trainy==0]
# detect outliers in the test set
yhat = lof_predict(model, trainX, testX)
# mark inliers 1, outliers -1
testy[testy == 1] = 1
testy[testy == 0] = -1
# calculate score
score = f1_score(testy, yhat, pos_label=-1)
print('F1 Score: %.3f' % score)

F1 Score: 0.095


The f1 score was even lower than with isolation forest.

# Adaboost

Sometimes different algorithms will help for an imbalanced data set.  Lets try boosting with adaboost which is simular to the Random Forest model.  This code and results are in the Random Forest notebook.

### easy ensemble 
which creates different data sets by selecting random samples of the majority class and balancing with the minority class and using adaboost.  

In [96]:
X =  data1.drop(columns=['Target']).values
y = data1['Target'].values

In [97]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.ensemble import EasyEnsembleClassifier


# define model
model = EasyEnsembleClassifier(n_estimators=10)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.3f' % np.mean(scores))


Mean ROC AUC: 0.591


In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)
model.fit(X_train, y_train) 

rfc_pred = model.predict(X_test)
print(confusion_matrix(y_test,rfc_pred))
print(classification_report(y_test,rfc_pred))

[[ 218  188]
 [1698 2183]]
              precision    recall  f1-score   support

           0       0.11      0.54      0.19       406
           1       0.92      0.56      0.70      3881

    accuracy                           0.56      4287
   macro avg       0.52      0.55      0.44      4287
weighted avg       0.84      0.56      0.65      4287



This model did not perform as well as others tested above.  